# PRÁCTICA 2A: Fine-tuning Eficiente con LoRA

**Objetivo**: Aprender técnicas de Parameter-Efficient Fine-Tuning (PEFT) con LoRA.

En este notebook exploraremos:
- **LoRA (Low-Rank Adaptation)**: Técnica de fine-tuning eficiente
- **PEFT**: Parameter-Efficient Fine-Tuning concepts
- **Comparación**: Entrenar solo ~1% de parámetros vs entrenar todo

Métricas a observar:
- Número de parámetros entrenables con LoRA
- Tiempo de entrenamiento
- Uso de memoria
- Accuracy final

## 1. Setup y Preparación del Dataset (5 min)

**Requisitos previos:**
- Python 3.8+
- Paquetes: `transformers`, `datasets`, `peft`, `evaluate`, `torch`
- Opcional: GPU con CUDA para acelerar (funciona también en CPU)

**Nota**: Este notebook usa un subset pequeño del dataset (1000 ejemplos) para demostración rápida.
Para resultados de producción, usa el dataset completo.

### Nota Importante sobre GPUs Múltiples

Si tienes múltiples GPUs y experimentas errores NCCL, ejecuta esto **ANTES** de abrir el notebook:
```bash
export CUDA_VISIBLE_DEVICES=0
```

O reinicia el kernel después de ejecutar la primera celda.


In [1]:
!pip install evaluate

In [2]:
# Imports
import torch
import numpy as np
import time
import os

# IMPORTANTE: Configurar para usar solo 1 GPU ANTES de importar transformers
# Esto previene errores NCCL en sistemas con múltiples GPUs
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Para debugging si es necesario

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    AutoModelForCausalLM # Añadido para generación de texto
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Número de GPUs visibles: {torch.cuda.device_count()}")

Usando dispositivo: cuda
GPU: Tesla T4
Memoria disponible: 15.83 GB
Número de GPUs visibles: 1


### 1.1 Cargar y Preparar el Dataset

Usaremos el dataset **IMDB** para clasificación de sentimiento (positivo/negativo).

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/data

In [3]:
!pip install datasets

In [4]:
# Cargar dataset IMDB
from datasets import load_dataset

print("Cargando dataset IMDB...")
dataset = load_dataset("yunfan-y/trump-qa")
def create_text_column(example):
    example['text'] = "Answer this question " + example['input']
    return example

print("Creando la columna 'text' en el dataset...")
# Apply the function to both train and test splits
dataset = dataset['train'].train_test_split(0.2)
dataset['train'] = dataset['train'].map(create_text_column)
dataset['test'] = dataset['test'].map(create_text_column)
print("Columna 'text' creada. Mostrando un ejemplo del dataset de entrenamiento:")
print(dataset['train'][0])

Cargando dataset IMDB...
Creando la columna 'text' en el dataset...


Map:   0%|          | 0/21256 [00:00<?, ? examples/s]

Map:   0%|          | 0/5314 [00:00<?, ? examples/s]

Columna 'text' creada. Mostrando un ejemplo del dataset de entrenamiento:
{'output': ' Trumpy really loves the Irish ', 'input': 'What are your favorite things about the Irish people? \n', 'instruction': 'You are impersonating Donald Trump. Answer the question.', '__index_level_0__': 22676, 'text': 'Answer this question What are your favorite things about the Irish people? \n'}


In [5]:
columns_to_remove = ['input', 'instruction', '__index_level_0__']

print("Eliminando columnas innecesarias...")
dataset['train'] = dataset['train'].remove_columns(columns_to_remove)
dataset['test'] = dataset['test'].remove_columns(columns_to_remove)
train_dataset = dataset['train']
test_dataset = dataset['test']
print("Columnas eliminadas. Nueva estructura del dataset:")
print(dataset)

Eliminando columnas innecesarias...
Columnas eliminadas. Nueva estructura del dataset:
DatasetDict({
    train: Dataset({
        features: ['output', 'text'],
        num_rows: 21256
    })
    test: Dataset({
        features: ['output', 'text'],
        num_rows: 5314
    })
})


### 1.2 Tokenización

In [6]:
# Cargar tokenizer
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"#google/gemma-3-1b-it
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token # Establecer el token de padding para modelos de generación como Gemma

# Función de tokenización para generación de texto
def tokenize_function(examples):
    # Concatenar 'text' (entrada) y 'output' (objetivo) para modelado de lenguaje causal
    # El modelo aprenderá a predecir la parte 'output' dada la parte 'text'.
    # Se añade el EOS token para separar claramente las partes, lo cual es una práctica común.
    combined_texts = [
        f"{text}{tokenizer.eos_token}{output}"
        for text, output in zip(examples["text"], examples["output"])
    ]

    tokenized_data = tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Para CausalLM, las etiquetas (labels) suelen ser los propios input_ids
    # para la predicción del siguiente token. El Trainer se encargará de desplazar
    # las etiquetas internamente.
    tokenized_data["labels"] = tokenized_data["input_ids"].copy()

    return tokenized_data

# Tokenizar datasets
print("Tokenizando datasets para generación de texto...")
# 'remove_columns' debe ahora eliminar tanto 'text' como 'output'
# ya que sus versiones tokenizadas (input_ids, attention_mask, labels) han sido creadas.
tokenized_train = dataset['train'].map(tokenize_function, batched=True, remove_columns=["text", "output"])
tokenized_test = dataset['test'].map(tokenize_function, batched=True, remove_columns=["text", "output"])

print("\nDatasets tokenizados:")
print(f"Train: {tokenized_train}")
print(f"Test: {tokenized_test}")

Tokenizando datasets para generación de texto...


Map:   0%|          | 0/21256 [00:00<?, ? examples/s]

Map:   0%|          | 0/5314 [00:00<?, ? examples/s]


Datasets tokenizados:
Train: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 21256
})
Test: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5314
})


# DataCollatorWithPadding

DataCollatorWithPadding es un componente de Hugging Face Transformers que prepara los batches de datos durante el entrenamiento. Veamos qué hace y sus alternativas:

¿Qué hace DataCollatorWithPadding?
Función principal: Aplica padding dinámico a las secuencias dentro de cada batch para que todas tengan la misma longitud.

Cómo funciona:

Recibe un batch de ejemplos tokenizados
Encuentra la secuencia más larga del batch
Rellena (pad) todas las demás secuencias a esa longitud
Crea las máscaras de atención automáticamente

In [7]:
# Data collator para padding dinámico
# Para CausalLM, es importante que el tokenizer del data collator tenga el pad_token configurado.
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,
    max_length=None,
    pad_to_multiple_of=None
)

# Para tareas de generación de texto (CausalLM), la métrica de 'accuracy' no es directamente aplicable.
# Se suelen usar métricas como perplexity, BLEU, ROUGE, etc., que son más complejas de implementar
# durante el entrenamiento con el Trainer. Por simplicidad, no definiremos compute_metrics aquí.
# El Trainer calculará la pérdida y métricas internas como la velocidad de entrenamiento.

# La función compute_metrics y accuracy_metric ya no son relevantes para este tipo de tarea.

## 3. LoRA: Parameter-Efficient Fine-Tuning

**Ahora veremos LoRA** - una técnica que permite entrenar modelos grandes con pocos recursos.
En lugar de entrenar todos los parámetros, añadimos matrices pequeñas de bajo rango.

### ¿Cómo funciona LoRA?

En lugar de actualizar directamente las matrices de peso $W$, LoRA añade dos matrices pequeñas $A$ y $B$:

$$W' = W + \Delta W = W + BA$$

Donde:
- $W \in \mathbb{R}^{d \times k}$ (matriz original, **congelada**)
- $B \in \mathbb{R}^{d \times r}$ (matriz entrenable)
- $A \in \mathbb{R}^{r \times k}$ (matriz entrenable)
- $r \ll \min(d, k)$ (rank pequeño, típicamente 4-16)

### Parámetros Clave de LoRA:

1. **`r` (rank)**: Dimensión del espacio de bajo rango
   - Valores típicos: 4, 8, 16, 32
   - Mayor rank = más parámetros entrenables = mejor capacidad pero más costoso
   - Recomendado: empezar con 8

2. **`lora_alpha`**: Factor de escalado
   - Típicamente `2*r` o `4*r`
   - Controla la magnitud de las actualizaciones de LoRA
   - No afecta el número de parámetros

3. **`target_modules`**: Qué capas modificar con LoRA
   - `["q_lin", "v_lin"]`: Solo query y value (común en transformers)
   - `["q_lin", "v_lin", "k_lin", "out_lin"]`: Todas las proyecciones de atención
   - Más módulos = más parámetros entrenables

4. **`lora_dropout`**: Dropout aplicado a las capas LoRA
   - Típicamente 0.05-0.1
   - Ayuda a prevenir overfitting

**Ventajas**:
- Solo entrenamos ~0.1-1% de parámetros
- Mucho más rápido y eficiente en memoria
- Ideal para múltiples tareas (puedes tener diferentes adaptadores LoRA)
- Fácil de compartir y combinar adaptadores


In [8]:
print("=" * 70)
print("ENTRENAMIENTO CON LORA (PEFT) PARA GENERACIÓN DE TEXTO")
print("=" * 70)

# Cargar modelo base para Causal Language Modeling (generación de texto)
# No se necesita num_labels para modelos de generación.
# Se utiliza bfloat16 para optimizar el uso de memoria y velocidad en GPUs compatibles (como las T4 en Colab).
model_lora = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16, # Use bfloat16 for Gemma (requires Ampere GPU or newer)
    device_map="auto" # Automatically map model to available devices
)

# Configurar LoRA para Causal Language Modeling
lora_config = LoraConfig(
    r=16,                                    # Rank: dimensión del espacio de bajo rango (4-32)
    lora_alpha=32,                          # Scaling factor: típicamente 2*r o 4*r
    # Target modules comunes para Gemma-3-4b-it en tareas de generación
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,                       # Dropout para regularización
    bias="none",                            # No entrenar bias adicionales
    task_type=TaskType.CAUSAL_LM            # Tipo de tarea: Causal Language Modeling
)

# Aplicar LoRA al modelo
model_lora = get_peft_model(model_lora, lora_config)

# Imprimir información del modelo
print("\nInformación del modelo con LoRA:")
model_lora.print_trainable_parameters()

# Contar parámetros manualmente para comparación
total_params_lora = sum(p.numel() for p in model_lora.parameters())
trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

print(f"\nRESUMEN:")
print(f"Parámetros totales: {total_params_lora:,}")
print(f"Parámetros entrenables: {trainable_params_lora:,}")
print(f"Porcentaje entrenable: {100 * trainable_params_lora / total_params_lora:.2f}%")
print(f"\n¡Solo entrenamos ~{100 * trainable_params_lora / total_params_lora:.2f}% de los parámetros del modelo!")

ENTRENAMIENTO CON LORA (PEFT) PARA GENERACIÓN DE TEXTO


`torch_dtype` is deprecated! Use `dtype` instead!



Información del modelo con LoRA:
trainable params: 18,464,768 || all params: 1,795,552,768 || trainable%: 1.0284

RESUMEN:
Parámetros totales: 1,795,552,768
Parámetros entrenables: 18,464,768
Porcentaje entrenable: 1.03%

¡Solo entrenamos ~1.03% de los parámetros del modelo!


In [9]:
# Configuración de entrenamiento
training_args_lora = TrainingArguments(
    output_dir="./results_lora_generation", # Se cambió el directorio de salida para distinguirlo
    num_train_epochs=10,              # Número de épocas completas de entrenamiento
    per_device_train_batch_size=2,   # Batch size para entrenamiento
    learning_rate=2e-4,              # Learning rate (típicamente más alto con LoRA: 1e-4 a 3e-4)
    weight_decay=0.01,               # Regularización L2
    eval_strategy="epoch",           # Evaluar al final de cada época
    save_strategy="no",              # No guardar checkpoints (para rapidez)
    logging_steps=50,                # Log cada 50 steps
    report_to="none",                # No reportar a wandb/tensorboard
    fp16=torch.cuda.is_available(),  # Mixed precision training si hay GPU
    # Para CausalLM, a menudo es crucial mantener columnas no utilizadas ya que las 'labels' pueden derivar de 'input_ids'.
    remove_unused_columns=False,
    # No se usa compute_metrics para CausalLM directamente en el Trainer a menos que se implemente una métrica específica de generación.
    # El Trainer calculará la pérdida por defecto.
)

# Crear Trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=None # Se eliminó, ya no es para clasificación
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
# Entrenar y medir tiempo/memoria
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Métricas de memoria
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    start_memory_lora = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"Memoria GPU inicial: {start_memory_lora:.2f} GB")

start_time = time.time()

print("\nIniciando entrenamiento con LoRA...")
print("   (Esto debería ser más rápido que full fine-tuning)\n")

trainer_lora.train()

end_time = time.time()
training_time_lora = end_time - start_time

# Reportar uso de memoria
if torch.cuda.is_available():
    peak_memory_lora = torch.cuda.max_memory_allocated() / 1e9  # GB
    current_memory_lora = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"\nREPORTE DE MEMORIA:")
    print(f"   Memoria inicial: {start_memory_lora:.2f} GB")
    print(f"   Memoria actual: {current_memory_lora:.2f} GB")
    print(f"   Memoria pico durante entrenamiento: {peak_memory_lora:.2f} GB")
    print(f"   Memoria adicional usada: {peak_memory_lora - start_memory_lora:.2f} GB")
else:
    peak_memory_lora = 0
    print(f"\nEjecutando en CPU (no hay métricas de memoria GPU)")

print(f"\nTIEMPO DE ENTRENAMIENTO:")
print(f"   {training_time_lora:.2f} segundos ({training_time_lora/60:.2f} minutos)")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 151646, 'pad_token_id': 151643}.


Memoria GPU inicial: 3.63 GB

Iniciando entrenamiento con LoRA...
   (Esto debería ser más rápido que full fine-tuning)



Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluar
print("\nEvaluando modelo con LoRA...")
eval_results_lora = trainer_lora.evaluate()

print("\n" + "=" * 70)
print("RESULTADOS - LORA (PEFT)")
print("=" * 70)
print(f"Accuracy: {eval_results_lora['eval_accuracy']:.4f}")
print(f"Loss: {eval_results_lora['eval_loss']:.4f}")
print(f"Parámetros entrenables: {trainable_params_lora:,} ({100 * trainable_params_lora / total_params_lora:.2f}%)")
print(f"Tiempo: {training_time_lora:.2f}s ({training_time_lora/60:.2f} min)")
if torch.cuda.is_available():
    print(f"Memoria pico: {peak_memory_lora:.2f} GB")
print("=" * 70)

print("\nLoRA permite hacer fine-tuning de modelos grandes eficientemente")
print("   Ideal para: recursos limitados, múltiples tareas, experimentación rápida")


### 3.1 Comparación Directa: Full Fine-tuning vs LoRA

Comparemos los resultados lado a lado:

In [ ]:
import pandas as pd

# Crear tabla comparativa
comparison_data = {
    'Métrica': [
        'Accuracy',
        'Loss',
        'Parámetros Entrenables',
        '% Parámetros',
        'Tiempo (segundos)',
        'Tiempo (minutos)',
        'Memoria Pico (GB)',
        'Speedup vs Full FT',
        'Ahorro Memoria vs Full FT'
    ],
    'Full Fine-tuning': [
        f"{eval_results_full['eval_accuracy']:.4f}",
        f"{eval_results_full['eval_loss']:.4f}",
        f"{trainable_params_full:,}",
        "100.00%",
        f"{training_time_full:.2f}",
        f"{training_time_full/60:.2f}",
        f"{peak_memory_full:.2f}" if torch.cuda.is_available() else "N/A (CPU)",
        "1.00x (baseline)",
        "0% (baseline)"
    ],
    'LoRA': [
        f"{eval_results_lora['eval_accuracy']:.4f}",
        f"{eval_results_lora['eval_loss']:.4f}",
        f"{trainable_params_lora:,}",
        f"{100 * trainable_params_lora / total_params_lora:.2f}%",
        f"{training_time_lora:.2f}",
        f"{training_time_lora/60:.2f}",
        f"{peak_memory_lora:.2f}" if torch.cuda.is_available() else "N/A (CPU)",
        f"{training_time_full/training_time_lora:.2f}x" if training_time_lora > 0 else "N/A",
        f"{100*(peak_memory_full - peak_memory_lora)/peak_memory_full:.1f}%" if torch.cuda.is_available() and peak_memory_full > 0 else "N/A"
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "=" * 80)
print("COMPARACIÓN: FULL FINE-TUNING vs LoRA")
print("=" * 80)
print(df_comparison.to_string(index=False))
print("=" * 80)

# Análisis de resultados
print("\nANÁLISIS:")
print(f"\n1. PRECISIÓN:")
acc_diff = eval_results_lora['eval_accuracy'] - eval_results_full['eval_accuracy']
if abs(acc_diff) < 0.01:
    print(f"   Accuracy similar entre ambos métodos (diferencia: {acc_diff:+.4f})")
else:
    print(f"   Accuracy diferente: {acc_diff:+.4f}")

print(f"\n2. EFICIENCIA DE PARÁMETROS:")
param_reduction = 100 * (1 - trainable_params_lora / trainable_params_full)
print(f"   LoRA entrena {param_reduction:.1f}% menos parámetros")
print(f"   Solo {trainable_params_lora:,} parámetros vs {trainable_params_full:,}")

print(f"\n3. VELOCIDAD:")
if training_time_lora > 0:
    speedup = training_time_full / training_time_lora
    time_saved = training_time_full - training_time_lora
    print(f"   LoRA es {speedup:.2f}x más rápido")
    print(f"   Ahorro de tiempo: {time_saved:.1f} segundos ({time_saved/60:.1f} minutos)")

if torch.cuda.is_available() and peak_memory_full > 0:
    print(f"\n4. MEMORIA:")
    memory_saved = peak_memory_full - peak_memory_lora
    memory_reduction = 100 * memory_saved / peak_memory_full
    print(f"   LoRA usa {memory_reduction:.1f}% menos memoria")
    print(f"   Ahorro: {memory_saved:.2f} GB")
    print(f"   Esto permite entrenar modelos más grandes en la misma GPU")

print(f"\nCONCLUSIÓN:")
print(f"   LoRA ofrece un excelente balance entre eficiencia y rendimiento.")
print(f"   Es ideal cuando tienes recursos limitados o necesitas entrenar rápido.")


## 4. Experimentando con LoRA

### 4.1 Efecto del Rank (r)

El parámetro más importante en LoRA es el **rank** ($r$). Veamos cómo afecta:

In [ ]:
# Comparar diferentes ranks
ranks = [4, 8, 16]

print("=" * 70)
print("COMPARACIÓN DE RANKS EN LORA")
print("=" * 70)

for r in ranks:
    # Crear modelo con LoRA
    model_temp = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )

    lora_config_temp = LoraConfig(
        r=r,
        lora_alpha=32,
        target_modules=["q_lin", "v_lin"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )

    model_temp = get_peft_model(model_temp, lora_config_temp)

    total_params = sum(p.numel() for p in model_temp.parameters())
    trainable_params = sum(p.numel() for p in model_temp.parameters() if p.requires_grad)

    print(f"\nRank r={r}:")
    print(f"  Parámetros entrenables: {trainable_params:,}")
    print(f"  Porcentaje: {100 * trainable_params / total_params:.2f}%")

    del model_temp

print("\n" + "=" * 70)
print("\nObservaciones:")
print("  - Ranks más altos = más parámetros entrenables")
print("  - Pero incluso con r=16, seguimos en ~2% de parámetros")
print("  - En la práctica: r=4-8 suele ser suficiente")
print("  - r muy alto puede causar overfitting en datasets pequeños")

## 5. Ejercicios Propuestos

### Ejercicio 1: Experimentar con Diferentes Ranks
Entrena modelos con r=4, r=8 y r=16. Compara:
- Accuracy final
- Tiempo de entrenamiento
- Memoria usada

### Ejercicio 2: Más Módulos Target
Prueba aplicar LoRA a más capas:
```python
target_modules=["q_lin", "v_lin", "k_lin", "out_lin"]
```
¿Cómo afecta al rendimiento?

### Ejercicio 3: Entrenar con Dataset Completo
Entrena con los 25K ejemplos completos del dataset IMDB:
- ¿Mejora el accuracy?
- ¿Cuánto tiempo toma?
- ¿Sigue siendo eficiente LoRA?

### Ejercicio 4: Otros Modelos
Prueba LoRA con otros modelos:
- `bert-base-uncased`
- `roberta-base`
- Modelos más grandes si tienes GPU

### Ejercicio 5: Datasets Diferentes
Aplica LoRA a otros tasks:
- `sst2` (sentimiento)
- `mrpc` (paráfrasis)
- `cola` (aceptabilidad gramatical)

## 6. Resumen

### ¿Qué aprendimos?

En este notebook comparamos **Full Fine-tuning** vs **LoRA (Low-Rank Adaptation)**:

**Full Fine-tuning**:
- Máxima capacidad de adaptación
- Alto uso de memoria (varios GB)
- Más lento
- Requiere más recursos

**LoRA (Low-Rank Adaptation)**:
- Entrena solo ~1% de parámetros
- **Eficiencia**: Entrenar solo ~1% de los parámetros del modelo
- **Velocidad**: 3-5x más rápido que full fine-tuning
- **Memoria**: Usa ~50-70% menos memoria
- **Modularidad**: Múltiples adaptadores para diferentes tareas

### Parámetros Críticos para Recordar:

| Parámetro | Valores Típicos | Efecto |
|-----------|----------------|--------|
| **r** (rank) | 4, 8, 16 | Mayor = más capacidad, más parámetros |
| **lora_alpha** | 16, 32, 64 | Típicamente 2*r o 4*r |
| **target_modules** | ["q_lin", "v_lin"] | Qué capas adaptar |
| **learning_rate** | 1e-4 a 3e-4 | Mayor que full fine-tuning |

### Cuándo usar LoRA:

**SÍ usar LoRA cuando:**
- Recursos limitados (GPU pequeña o CPU)
- Necesitas entrenar rápidamente
- Múltiples tareas/dominios con el mismo modelo base
- Experimentación rápida con diferentes configuraciones

**NO usar LoRA cuando:**
- Tienes recursos ilimitados y necesitas máximo rendimiento
- La tarea es muy diferente del pretraining
- Dataset extremadamente grande y específico

### Próximos Pasos:

1. **Experimenta** con diferentes ranks (r=4, 8, 16, 32)
2. **Prueba** más target_modules
3. **Compara** con full fine-tuning en tu tarea específica
4. **Explora** QLoRA para modelos aún más grandes


## 7. Recursos Adicionales

### Papers Importantes:
- **LoRA**: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- **QLoRA**: [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)
- **Prefix Tuning**: [Prefix-Tuning: Optimizing Continuous Prompts](https://arxiv.org/abs/2101.00190)

### Librerías:
- **Hugging Face PEFT**: https://github.com/huggingface/peft
- **Documentación PEFT**: https://huggingface.co/docs/peft

### Investiga otras técnicas PEFT:
- **Adapter Layers**: Añadir capas pequeñas entre layers
- **Prompt Tuning**: Solo entrenar los embeddings del prompt
- **QLoRA**: LoRA + 4-bit quantization

Creado por Jorge Dueñas Lerín
jorge.duenas.lerin@upm.es